![](../images/xor_and_or_nand.png)

## Minimum Integer Set Algorithm
- construct linear program that solves the constraints from individual gates

In [13]:
# find the size of the minimum set of integer numbers for representing AND, OR, NAND with a neuron and sigmoid activation function
import gurobipy as g
import itertools
import math


NUMBER_OF_INTEGERS = 3
NUMBER_OF_HELPER_VARIABLES = pow(NUMBER_OF_INTEGERS, NUMBER_OF_INTEGERS)
# Big-M
M = 100

model = g.Model()

# define number of variables
int_set = model.addVars(NUMBER_OF_INTEGERS, vtype=g.GRB.INTEGER, lb=-15, ub=15, name="x")


# generate all possible permutations with repetition of integer variables
numbers = list(range(NUMBER_OF_INTEGERS))
all_permutations = list(itertools.product(numbers, repeat=NUMBER_OF_INTEGERS))
print(all_permutations)

helper_AND_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b1")
# helper_OR_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b2")
# helper_NAND_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b3")
helper_NOR_variables = model.addVars(NUMBER_OF_HELPER_VARIABLES, vtype=g.GRB.BINARY, name="b4")

# AND constraints
for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_AND_variables[index]) ,f"AND 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_AND_variables[index]) ,f"AND 4.{permutation}")


model.addConstr(g.quicksum(helper_AND_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneANDGroup")

# # OR constraints
# for index, permutation in enumerate(all_permutations):
#     model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_OR_variables[index]) ,f"OR 1.{permutation}")
#     model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 2.{permutation}")
#     model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 3.{permutation}")
#     model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_OR_variables[index]) ,f"OR 4.{permutation}")


# model.addConstr(g.quicksum(helper_OR_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneORGroup")

# # NAND constraints
# for index, permutation in enumerate(all_permutations):
#     model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 1.{permutation}")
#     model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 2.{permutation}")
#     model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NAND_variables[index]) ,f"NAND 3.{permutation}")
#     model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NAND_variables[index]) ,f"NAND 4.{permutation}")

# model.addConstr(g.quicksum(helper_NAND_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneNANDGroup")

for index, permutation in enumerate(all_permutations):
    model.addConstr(0*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] >= 5 - M * (1 - helper_NOR_variables[index]) ,f"NOR 1.{permutation}")
    model.addConstr(0*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 2.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 0*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 3.{permutation}")
    model.addConstr(1*int_set[permutation[0]] + 1*int_set[permutation[1]] + int_set[permutation[2]] <= -5 + M * (1 - helper_NOR_variables[index]) ,f"NOR 4.{permutation}")

model.addConstr(g.quicksum(helper_NOR_variables[i] for i in range(NUMBER_OF_HELPER_VARIABLES)) >= 1, "AtLeastOneNANDGroup")

# Optimize
model.optimize()


'''
The viable integers for representing the set of logic gates
'''
for index in range(NUMBER_OF_INTEGERS):
    print(f"The integer {index} is {int_set[index].x}")

'''
the index of where helper variable is 1 denotes represents the specific permutation, 
which is viable for representing representing the logic gate
'''
for index in range(NUMBER_OF_HELPER_VARIABLES):
    if (helper_AND_variables[index].x > 0.1):
        print(f"The permutation for AND is {all_permutations[index]}")
    # if (helper_OR_variables[index].x > 0.1):
    #     print(f"The permutation for OR is {all_permutations[index]}")
    # if (helper_NAND_variables[index].x > 0.1):
    #     print(f"The permutation for NAND is {all_permutations[index]}")
    if (helper_NOR_variables[index].x > 0.1):
        print(f"The permutation for NAND is {all_permutations[index]}")


[(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2)]
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Core(TM) i7-10875H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 218 rows, 57 columns and 618 nonzeros
Model fingerprint: 0xe355df86
Variable types: 0 continuous, 57 integer (54 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 1e+02]
Presolve removed 132 rows and 30 columns
Presolve time: 0.00s
Presolved: 86 rows, 27 columns, 264 nonzeros
Variable types: 0 continuous, 27 integer (24

In [52]:
# maximize distances
from itertools import product

# Define the elements to permute
elements = [0, 1, 2]

# Define the length of each permutation
permutation_length = 3

NUMBER_OF_INTEGERS = 3
NUMBER_OF_HELPER_VARIABLES = pow(NUMBER_OF_INTEGERS, NUMBER_OF_INTEGERS)
print(NUMBER_OF_HELPER_VARIABLES)
# Generate permutations with repeating elements
permutations = list(product(elements, repeat=permutation_length))

i = 0
# Print the permutations
for perm in permutations:
    print(perm)
    i += 1

print(i)

27
(0, 0, 0)
(0, 0, 1)
(0, 0, 2)
(0, 1, 0)
(0, 1, 1)
(0, 1, 2)
(0, 2, 0)
(0, 2, 1)
(0, 2, 2)
(1, 0, 0)
(1, 0, 1)
(1, 0, 2)
(1, 1, 0)
(1, 1, 1)
(1, 1, 2)
(1, 2, 0)
(1, 2, 1)
(1, 2, 2)
(2, 0, 0)
(2, 0, 1)
(2, 0, 2)
(2, 1, 0)
(2, 1, 1)
(2, 1, 2)
(2, 2, 0)
(2, 2, 1)
(2, 2, 2)
27


### min weight set for AND and NOR
| GATE | BIAS | WEIGHT1 | WEIGHT2 |
| AND  |   -6  |    4    |    4    |
| NOR  |   4  |    -6    |    -6    |


## Possible activation functions
- sigmoid
- threshold function
- hyperbolic tanget

In [ ]:
# global imports
import torch

In [42]:
import math

def stable_sigmoid(x):

    if x >= 0:
        z = math.exp(-x)
        sig = 1 / (1 + z)
        return sig
    else:
        z = math.exp(x)
        sig = z / (1 + z)
        return sig

stable_sigmoid(3)

0.9525741268224334

# Algorithms

## training ternary neural networks by rectified L2 regularization
use it for.:
### min weight set for AND and NOR
| GATE | BIAS | WEIGHT1 | WEIGHT2 |
| AND  |   -6  |    4    |    4    |
| NOR  |   4  |    -6    |    -6    |

### simplest architecture
![](../images/simplest_xor_architecture.png)

In [113]:
import numpy as np

def sigmoid (x):
	return 1/(1 + np.exp(-x))

def sigmoid_derivative(x):
	return x * (1 - x)

# inputs
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
expected_output = np.array([[0],[1],[1],[0]])

# hyperparameters
epochs = 10000
lr = 0.1
penalty_coefficient = 0.03
dual_parameters = [4, -6]
threshold = -1

# initialize weigths and biases
inputLayerNeurons, hiddenLayerNeurons, outputLayerNeurons = 2,2,1
hidden_weights = np.random.uniform(low=-6, high=4, size=(inputLayerNeurons,hiddenLayerNeurons))
hidden_bias =np.random.uniform(low=-6, high=4, size=(1,hiddenLayerNeurons))
output_weights = np.random.uniform(low=-6, high=4, size=(hiddenLayerNeurons,outputLayerNeurons))
output_bias = np.random.uniform(low=-6, high=4, size=(1,outputLayerNeurons))

#Training algorithm
for _ in range(epochs):
	#Forward Propagation
	hidden_layer_activation = np.dot(inputs,hidden_weights)
	hidden_layer_activation += hidden_bias
	hidden_layer_output = sigmoid(hidden_layer_activation)

	output_layer_activation = np.dot(hidden_layer_output,output_weights)
	output_layer_activation += output_bias
	predicted_output = sigmoid(output_layer_activation)

	#Backpropagation
	error = expected_output - predicted_output
	d_predicted_output = error * sigmoid_derivative(predicted_output)

	error_hidden_layer = d_predicted_output.dot(output_weights.T)
	d_hidden_layer = error_hidden_layer * sigmoid_derivative(hidden_layer_output)

	#Updating Weights and Biases
	output_weights += hidden_layer_output.T.dot(d_predicted_output) * lr
	output_bias += np.sum(d_predicted_output,axis=0,keepdims=True) * lr
	hidden_weights += inputs.T.dot(d_hidden_layer) * lr
	hidden_bias += np.sum(d_hidden_layer,axis=0,keepdims=True) * lr

	x = 0
	# todo: do this properly by backpropagation with matrix operations
	if(epochs > 20):
		for i, weight in enumerate(output_weights):
			if weight > -1:
				output_weights[i] = weight - 2*penalty_coefficient*(weight-4)
			else:
				output_weights[i] = weight - 2*penalty_coefficient*(weight+6)

		if output_bias[0] > -1:
			output_bias[0] = output_bias[0] - 2*penalty_coefficient*(output_bias[0]-4)
		else:
			output_bias[0] = output_bias[0] - 2*penalty_coefficient*(output_bias[0]+6)

		for i, weights in enumerate(hidden_weights):
			for j, weight in enumerate(weights):
				if weight > -1:
					hidden_weights[i,j] = weight - 2*penalty_coefficient*(weight-4)
				else:
					hidden_weights[i,j] = weight - 2*penalty_coefficient*(weight+6)

		for i, weights in enumerate(hidden_bias):
			for j, weight in enumerate(weights):
				if weight > -1:
					hidden_bias[i,j] = weight - 2*penalty_coefficient*(weight-4)
				else:
					hidden_bias[i,j] = weight - 2*penalty_coefficient*(weight+6)



# results
print("Final hidden weights: ",end='')
print(*hidden_weights)
print("Final hidden bias: ",end='')
print(*hidden_bias)
print("Final output weights: ",end='')
print(*output_weights)
print("Final output bias: ",end='')
print(*output_bias)
print("\nOutput from neural network after 10,000 epochs: ",end='')
print(*predicted_output)

Final hidden weights: [-4.91343871 -1.47922463] [4.00612084 1.58242818]
Final hidden bias: [-2.84768601  1.454252  ]
Final output weights: [4.03872877] [-6.06001498]
Final output bias: [3.95049173]

Output from neural network after 10,000 epochs: [0.323194] [0.77569961] [0.72262805] [0.27666736]
